In [1]:
!wget --user=zlyuac --ask-password https://course.cse.ust.hk/comp2211/assignments/pa1/data/20ng_train_dataset.npz
!wget --user=zlyuac --ask-password https://course.cse.ust.hk/comp2211/assignments/pa1/data/20ng_test_dataset.npz
!wget --user=zlyuac --ask-password https://course.cse.ust.hk/comp2211/assignments/pa1/data/20ng_train_labels.npy
!wget --user=zlyuac --ask-password https://course.cse.ust.hk/comp2211/assignments/pa1/data/20ng_test_labels.npy

Password for user ‘zlyuac’: 
--2022-10-19 16:28:16--  https://course.cse.ust.hk/comp2211/assignments/pa1/data/20ng_train_dataset.npz
Resolving course.cse.ust.hk (course.cse.ust.hk)... 143.89.41.176
Connecting to course.cse.ust.hk (course.cse.ust.hk)|143.89.41.176|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="Enter Your CSD PC/Unix Password"
Reusing existing connection to course.cse.ust.hk:443.
HTTP request sent, awaiting response... 200 OK
Length: 3253235 (3.1M)
Saving to: ‘20ng_train_dataset.npz.2’

20ng_train_dataset. 100%[===================>]   3.10M  2.10MB/s    in 1.5s    

2022-10-19 16:28:19 (2.10 MB/s) - ‘20ng_train_dataset.npz.2’ saved [3253235/3253235]

Password for user ‘zlyuac’: 
--2022-10-19 16:28:27--  https://course.cse.ust.hk/comp2211/assignments/pa1/data/20ng_test_dataset.npz
Resolving course.cse.ust.hk (course.cse.ust.hk)... 143.89.41.176
Connecting to course.cse.ust.hk (course.cse.ust.hk)|143.89.41.

In [2]:
import numpy as np

In [3]:
class NaiveBayesClassifier:
  def __init__(self, train_dataset, test_dataset, train_labels, test_labels):
    self.train_dataset = train_dataset
    self.test_dataset = test_dataset
    self.train_labels = train_labels
    self.test_labels = test_labels

  def build_training_delta_matrix(self):
    #TODO
    deltas = np.zeros((self.train_dataset.shape[0],np.amax(self.train_labels)+1))
    #number of samples rows, number of classes cols
    for i in range(self.train_dataset.shape[0]):
       deltas[i][self.train_labels[i]] = 1 
    return deltas

  def estimate_class_probabilities(self):
    #TODO
    delta = self.build_training_delta_matrix()
    numerator = 1 + np.sum(delta,axis = 0)
    denominator = np.amax(self.train_labels)+1 + self.train_dataset.shape[0]
    class_prob = numerator / denominator
    return class_prob

  def estimate_word_probabilities(self):
    #TODO
    #word_prob: size of voca rows, number of classes cols
    delta = self.build_training_delta_matrix()
    # (dataset)number of documents rows, size of voca cols,element(Xik,frequency of a word in a document)
    numerator = 1 + self.train_dataset.T @ delta #for one class, sum of frequency of a word in all documents.\
    # size: size of voca * num of classes

    #sum of frequency of all words, in one class
    denominator = self.train_dataset.shape[1] + np.sum(self.train_dataset.T @ delta, axis = 0)

    word_prob = numerator / denominator
    return word_prob

  def predict(self):
    #TODO
    log_word_prob = np.log(self.estimate_word_probabilities())
    temp = np.log(self.estimate_class_probabilities()) + np.sum(self.train_dataset @ log_word_prob, axis = 1)
    test_predict = np.amax(temp, axis = 1)
    return test_predict

In [4]:
  def generate_confusion_matrix(test_predict, test_labels):
    #TODO
    TP = np.zeros(np.amax(test_predict) + 1)
    for i in range(np.amax(test_predict)):
      TP = np.count(test_labels[test_predict == i] == test_predict) 
      TN = np.count(test_labels[test_predict == i] != test_predict)
      FP = np.count(test_predict[test_labels != i] == i)
      FN = np.count(test_predict[test_labels == i] != i)
    return TP, TN, FP, FN

In [5]:
  def calculate_precision(test_predict, test_labels, self):
    #TODO
    TP, TN, FP, FN = self.generate_confusion_matrix(test_predict, test_labels)
    numerator = np.sum(TP)
    demoninator = np.sum(TP + FP)
    precision = numerator / demoninator
    return precision

In [6]:
  def calculate_recall(test_predict, test_labels, self):
    #TODO
    TP, TN, FP, FN = self.generate_confusion_matrix(test_predict, test_labels)
    numerator = np.sum(TP)
    demoninator = np.sum(TP + FN)
    recall = numerator / demoninator
    return recall

In [7]:
  def calculate_micro_f1(test_predict, test_labels, self):
    #TODO
    p = self.calculate_precision(test_predict, test_labels)
    R = self.calculate_recall(test_predict, test_labels)
    micro_f1 = 2 * P * R / (P + R)
    return micro_f1

In [8]:
  def calculate_macro_f1(test_predict, test_labels, self):
    #TODO
    num_classes = np.amax(self.train_labels)
    num_test_documents = train_dataset.shape[0]
    TP, TN, FP, FN = self.generate_confusion_matrix(test_predict, test_labels)
    R, P = np.zeros(num_test_documents, num_classes)
    R = TP / (TP + FN)
    P = TP / (TP + FN)
    macro_f1 = 1/num_classes * np.sum(2 * P * R / (P + R))
    return macro_f1

# Optional Task: Test Run
Use all the previously defined functions in Tasks 1 and 2 to perform the Naive Bayes text classifier on our 20 Newsgroups dataset. Feel free to modify this code cell for your own testing and debugging purposes, which will not be graded.

In [9]:
import scipy.sparse as sparse

if __name__ == '__main__':
  train_dataset = sparse.load_npz("20ng_train_dataset.npz")
  test_dataset = sparse.load_npz("20ng_test_dataset.npz")
  train_dataset = train_dataset.toarray()
  test_dataset = test_dataset.toarray()
  train_labels = np.load("20ng_train_labels.npy")
  test_labels = np.load("20ng_test_labels.npy")

  #TODO Optional

# References
Kamal Nigam, Andrew McCallum, and Tom Mitchell. 2006. Semi-supervised text classification using EM. Semi-Supervised Learning (2006), 33–56.